In [18]:
import pandas as pd
import re
import numpy as np
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV


In [2]:
import tweepy,csv,time

ACCESS_TOKEN = '938493239853813762-tlImlRTGU6RraqmBQQQQ3K5klFmHwvG'
ACCESS_SECRET = 'Ga79wDf0YK9YtYc21XEfBH10L0wAttnDL5wQXVT7oNj1J'
CONSUMER_KEY = 'zHK9haWtcpGWHUq12fEMn3Fyw'
CONSUMER_SECRET = 'jIZj1D7jPfXPGSIQTTEGMPgXhIR7lEqyBXRUnudcgpvPDc2wzm'

auth = tweepy.OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN,ACCESS_SECRET)

api = tweepy.API(auth)

In [3]:
# Tool to delete unicode codes
def unicodetoascii(text):
    TEXT = (text.
    		replace('\\xe2\\x80\\x99', "'").
            replace('\\xc3\\xa9', 'e').
            replace('\\xe2\\x80\\x90', '-').
            replace('\\xe2\\x80\\x91', '-').
            replace('\\xe2\\x80\\x92', '-').
            replace('\\xe2\\x80\\x93', '-').
            replace('\\xe2\\x80\\x94', '-').
            replace('\\xe2\\x80\\x94', '-').
            replace('\\xe2\\x80\\x98', "'").
            replace('\\xe2\\x80\\x9b', "'").
            replace('\\xe2\\x80\\x9c', '"').
            replace('\\xe2\\x80\\x9c', '"').
            replace('\\xe2\\x80\\x9d', '"').
            replace('\\xe2\\x80\\x9e', '"').
            replace('\\xe2\\x80\\x9f', '"').
            replace('\\xe2\\x80\\xa6', '...').#
            replace('\\xe2\\x80\\xb2', "'").
            replace('\\xe2\\x80\\xb3', "'").
            replace('\\xe2\\x80\\xb4', "'").
            replace('\\xe2\\x80\\xb5', "'").
            replace('\\xe2\\x80\\xb6', "'").
            replace('\\xe2\\x80\\xb7', "'").
            replace('\\xe2\\x81\\xba', "+").
            replace('\\xe2\\x81\\xbb', "-").
            replace('\\xe2\\x81\\xbc', "=").
            replace('\\xe2\\x81\\xbd', "(").
            replace('\\xe2\\x81\\xbe', ")")
                 )
    return TEXT

#Reading CSV file
dataset=pd.read_csv('tweets3.csv', sep=',',quotechar='"',header=None,)
#Replacing Unicode characters
dataset[2]=dataset[2].apply(unicodetoascii)
#Deleting Urls
delete_urls = lambda x:re.sub(r'http\S+', '', x)
dataset[2] = dataset[2].apply(delete_urls)
#Deleting "
delete_quotes = lambda x:re.sub(r'"', '', x)
dataset[2] = dataset[2].apply(delete_quotes)
#Deleting b
delete_b = lambda x:re.sub(r'b\'', '', x)
dataset[2] = dataset[2].apply(delete_b)

#Text cleaning
def clean (X):
    X=delete_urls(X)
    X=delete_quotes(X)
    X=delete_b(X)
    return X
#Datasets
X=dataset[2].values
Y=dataset[3].values


In [4]:
#Data split
x_train, x_test ,y_train,y_test = train_test_split(X,Y, test_size=0.3)

In [44]:
#Machine Learning NB
text_clf_nb = Pipeline([('vect', CountVectorizer(stop_words='english')),('tfidf', TfidfTransformer()),('clf', MultinomialNB()),])

#Grid search NB
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],'tfidf__use_idf': (True, False),'clf__alpha': (1e-2, 1e-3)}
gs_clf = GridSearchCV(text_clf_nb, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(x_train,y_train)

#Fitting with best hyperparameters
text_clf_nb = gs_clf.best_estimator_.fit(x_train, y_train)

# Performance
y_predicted = text_clf_nb.predict(x_test)
print("Accuracy :" + str(np.mean(y_predicted == y_test))+"%")
# 80 % accuracy

Accuracy :0.800179390074%


In [45]:
#Machine Learning SVM
text_clf_svm = Pipeline([('vect', CountVectorizer(stop_words='english')),
                   ('tfidf', TfidfTransformer()),
                   ('clf-svm', LinearSVC()) ])
#Grid search SVM
powers=np.array([2**-15,2**-13,2**-11,2**-9,2**-7])
parameters_svm = {'vect__ngram_range': [(1, 1), (1, 2)],'tfidf__use_idf': (True, False),'clf-svm__C': powers}
gs_clf_svm = GridSearchCV(text_clf_svm, parameters_svm, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(x_train,y_train)

#Fitting with best hyperparameters
text_clf_svm = gs_clf_svm.best_estimator_.fit(x_train,y_train)

# Performance
predicted_svm = text_clf_svm.predict(x_test)
print("Accuracy :"+ str(np.mean(predicted_svm == y_test))+"%")
# 73,9 % accuracy

Accuracy :0.739685070759%


In [50]:
#classifiers = text_clf_svm or text_clf_nb

# Input : text + pipeline(fitted) , output : prediction + accuracy (works only with NB classifier)
def tweet_prediction (txt,func):
    iter = np.array([txt])
    if (func==text_clf_nb):
        if (func.predict(iter)[0] == 1):
            print("This tweet is recognized as democrat with a "+str(round(func.predict_proba(iter)[0][1]*100,2))+" % certainty")
            return 1
        else:
            print("This tweet is recognized as republican with a "+str(round(func.predict_proba(iter)[0][0]*100,2))+" % certainty")
            return -1
    else:
        if (func.predict(iter)[0] == 1):
            print("This tweet is recognized as democrat.")
            return 1
        else:
            print("This tweet is recognized as republican.")
            return -1
    
tweet_prediction("Trump",text_clf_nb)
tweet_prediction("Trump",text_clf_svm)

This tweet is recognized as democrat with a 79.99 % certainty
This tweet is recognized as democrat.


1

In [66]:
#classifiers = text_clf_svm or text_clf_nb

# Input : tweeter account + pipeline (fitted), output : prediction 
api = tweepy.API(auth)
def account_prediction (acc,func,verbose=None,count=None):
    iter=np.array([])
    if (count==None):
        count=10000
    tweets = api.user_timeline(acc,count=count)
    for tweet in tweets:
        if ('RT @' not in tweet.text):
            iter=np.append([clean(tweet.text)],iter)
            
    result = func.predict(iter)
    ratio = np.mean(result)
    #print (ratio)
    if ratio <0 :
        if (verbose==None):
            print("The account @"+acc+" is recognized as a republican account.")
        return -1
    else:
        if (verbose==None):
            print("The account @"+acc+" is recognized as a democrat account.")
        return 1
account_prediction("RepMiaLove",text_clf_nb)
#account_prediction("RepMiaLove",text_clf_svm)


The account @RepMiaLove is recognized as a republican account.


-1

In [75]:
#Accuracy on accounts of US representatives 
#For reason of computing time, we only test accuracy for 100 accounts based on 50 last tweets 
def account_accuracy (classifier):
    start_time = time.time()
    acc_number=50
    last_tweets=50
    y=np.array([])
    predicted=np.array([])
    for member in tweepy.Cursor(api.list_members, 'thedemocrats', 'house-democrats').items(acc_number):
        y=np.append(y,[1])
        predicted=np.append(predicted,account_prediction(member.screen_name,classifier,False,last_tweets))
    for member in tweepy.Cursor(api.list_members, 'housegop', 'house-republicans').items(acc_number):
        y=np.append(y,[-1])
        predicted=np.append(predicted,account_prediction(member.screen_name,classifier,False,last_tweets))
    print("--- %s seconds ---" % (time.time() - start_time))
    print("Accuracy :" + str(np.mean(predicted==y)))

print("Classifier : NB")
account_accuracy(text_clf_nb)
print("Classifier : SVM")
account_accuracy(text_clf_svm)

Classifier : NB
--- 215.87572479248047 seconds ---
Accuracy :0.99
Classifier : SVM
--- 198.85703444480896 seconds ---
Accuracy :0.87
